Stochastic Military Port Selection Problem
==========================================

In [1]:
# Title: Stochastic Military Port Selection Problem
# Description: This extends the 
# Author: Lennon Conson

Load Data inport Packages
-------------------------

In [2]:
import csv
from datetime import date

Load Data
---------

In [3]:
# Import SPOEs/Transshipment Nodes
def transshipment_nodes_import(file_name):
    transshipment_nodes = set()
    with open(file_name, newline='') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            transshipment_nodes.add(row[0])
    return transshipment_nodes

transshipment_nodes = transshipment_nodes_import('daily-port-processing-capabilities.csv')

# Import shipments
def shipments_import(file_name):
    shipments = set()
    with open(file_name, newline='') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            shipments.add(row[0])
    return shipments

shipments = shipments_import('packages.csv')

# Import ships
def ships_import(file_name):
    ships = set()
    with open(file_name, newline='') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            ships.add(row[0])
    return ships

ships = ships_import('ships.csv')

Initialize Pyomo
--------------

In [4]:
import pyomo.environ as pyo
model = pyo.ConcreteModel()

Define Decision Variable
------------------------

In [5]:
# First Stage
# Which SPOEs to open
# A binary decision variable resenting if a SPOE (transshipment node) $j \in J$ opened during the first-stage decision. 
# A value of 0 represents a closed transshipment node, while a value of 1 represents an open transshipment node.
model.x = pyo.Var(transshipment_nodes, within=pyo.Binary)

# Second Stage
# Inflow
# Inflow Time
# A continuous non-negative decision variable that indicates at what time a shipment, k ∈ K, departs the origin.
model.t_inflow = pyo.Var(shipments, within=pyo.NonNegativeReals)

# Inflow Transshipment Node
# A categorical decision variable resenting which SPOE a shipment , k ∈ K, will flow through
model.y_inflow = pyo.Var(shipments, within=transshipment_nodes)

# Inflow Transportation mode
# A binary decision variable resenting which mode of travel that shipment , k ∈ K,
# is used to reach the SPOE. A 0 represents rail transport, while 1 represents
# road transport
model.z_inflow = pyo.Var(shipments, within=pyo.Binary)

# Outflow
# Outflow Time
# A continuous non-negative decision variable that indicates at what time a ship, l ∈ L, departs the a SPOE
model.t_outflow = pyo.Var(ships, within=pyo.NonNegativeReals)

# TODO Maybe unneeded
# Outflow Transshipment Node
# A categorical decision variable resenting which SPOE a ship, l ∈ L, will travel to.
# model.y_outflow = pyo.Var(ships, within=transshipment_nodes)

# Outflow Transportation Mode
# A categorical decision variable resenting which ship a shipment, k ∈ K, is utilizing
# to reach the destination.
model.z_outflow = pyo.Var(shipments, within=pyo.Binary)

Contraints
----------